In [4]:
%use bash
wget -O jq https://github.com/stedolan/jq/releases/latest/download/jq-linux64
chmod +x jq

--2025-06-03 16:52:12--  https://github.com/stedolan/jq/releases/latest/download/jq-linux64
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/jqlang/jq/releases/latest/download/jq-linux64 [following]
--2025-06-03 16:52:12--  https://github.com/jqlang/jq/releases/latest/download/jq-linux64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jqlang/jq/releases/download/jq-1.8.0/jq-linux64 [following]
--2025-06-03 16:52:12--  https://github.com/jqlang/jq/releases/download/jq-1.8.0/jq-linux64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/5101141/656b168e-a425-4028-87bb-947aca82a3f1?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Cre

In [5]:
%use bash
# Set output file name
OUTPUT_FILE="beer_places_world.json"

read -r -d '' QUERY << EOM
[out:json][timeout:1800];
(
  node["amenity"="pub"];
  node["amenity"="bar"];
  node["craft"="brewery"];
  node["cuisine"~"beer"];
  way["amenity"="pub"];
  way["amenity"="bar"];
  way["craft"="brewery"];
  relation["amenity"="pub"];
  relation["amenity"="bar"];
  relation["craft"="brewery"];
);
out body;
>;
out skel qt;
EOM

ENCODED_QUERY=$(echo "$QUERY" | ./jq -sRr @uri)
curl -o "$OUTPUT_FILE" "https://overpass-api.de/api/interpreter?data=$ENCODED_QUERY"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  173M    0  173M    0     0   240k      0 --:--:--  0:12:19 --:--:-- 1402k


In [9]:
%use bash
npm install osmtogeojson

(################⠂⠂) ⠇ reify:minimist: http fetch GET 200 https://registry.npmjnpm WARN deprecated @xmldom/xmldom@0.8.3: this version has critical issues, please update to the latest version
(##################) ⠼ reify:htmlparser2: http fetch GET 200 https://registry.ndules/@xmldo
added 35 packages, and audited 36 packages in 3s

4 packages are looking for funding
  run `npm fund` for details

2 critical severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.
npm notice 
npm notice New major version of npm available! 8.19.2 -> 11.4.1
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.4.1
npm notice Run npm install -g npm@11.4.1 to update!
npm notice 



In [1]:
%use javascript
const fs = require("fs");
const osmtogeojson = require("osmtogeojson");

const osmData = JSON.parse(fs.readFileSync("beer_places_world.json"));
const geojson = osmtogeojson(osmData);
fs.writeFileSync("beer_places_world.geojson", JSON.stringify(geojson, null, 2));
console.log("✅ Converted to GeoJSON: beer_places.geojson");


✅ Converted to GeoJSON: beer_places.geojson


In [2]:
%use python3
pip install geopandas geopy tqdm shapely numpy==1.23


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 9.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 8.9 MB/s eta 0:00:0000:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
  Attempting uninstall: geopandas
    Found existing installation: geopandas 1.1.0
    Uninstalling geopandas-1.1.0:
      Successfully uninstalled geopandas-1.1.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%use python3
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from tqdm import tqdm
import pandas as pd

# Load GeoJSON data
gdf = gpd.read_file("beer_places_world.geojson")

# Ensure coordinates are WGS84
gdf = gdf.to_crs(epsg=4326)

# Initialize geocoder
geolocator = Nominatim(user_agent="beer_density_calc")
tqdm.pandas()

# Reverse geocode to get city name
def get_city_name(point):
    try:
        location = geolocator.reverse((point.y, point.x), exactly_one=True, timeout=10)
        address = location.raw.get("address", {})
        return address.get("city") or address.get("town") or address.get("village") or None
    except:
        return None

# Add city column
print("🔄 Reverse geocoding cities...")
gdf["city"] = gdf.geometry.progress_apply(get_city_name)

# Drop rows without city
gdf = gdf.dropna(subset=["city"])

# Group by city and count
city_counts = gdf.groupby("city").size().reset_index(name="num_beer_places")

# Estimate area and density using convex hull
density_data = []

for city, group in gdf.groupby("city"):
    if len(group) < 3:
        continue  # not enough points to form a polygon
    try:
        hull = group.unary_union.convex_hull
        # Approximate area in km² using crude lat/lon scale (1° ~ 111km)
        area_km2 = hull.area * (111**2)
        density = len(group) / area_km2
        density_data.append((city, len(group), area_km2, density))
    except Exception as e:
        print(f"⚠️ Skipping {city}: {e}")

density_df = pd.DataFrame(density_data, columns=["city", "num_beer_places", "area_km2", "beer_per_km2"])

# Sort results
top_count = city_counts.sort_values(by="num_beer_places", ascending=False).head(10)
top_density = density_df.sort_values(by="beer_per_km2", ascending=False).head(10)

# Output
print("\n🏆 Top 10 Cities by Number of Beer Places:")
print(top_count)

print("\n🔥 Top 10 Cities by Beer Place Density (per km²):")
print(top_density)


/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
Skipping field contact:phone:description: unsupported OGR type: 10
